<a href="https://colab.research.google.com/github/SinaRampe/pubmed_downloader/blob/main/pubmed_downloader_with_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### If from an already performed PubMed search the results were saved as csv 

In [124]:
!pip install metapub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [146]:
import pandas as pd
import numpy as np
from metapub import FindIt
import requests
import shutil
import os

In [147]:
csv_data = "csv-cpic-set"
df = pd.read_csv(f'{csv_data}.csv')  

In [148]:
save_directory = f"pdfs_{csv_data}"

if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [149]:
df.head(1)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,32189324,Clinical Pharmacogenetics Implementation Conso...,"Theken KN, Lee CR, Gong L, Caudle KE, Formea C...",Clin Pharmacol Ther. 2020 Aug;108(2):191-200. ...,Theken KN,Clin Pharmacol Ther,2020,2020/03/20,PMC8080882,NIHMS1687646,10.1002/cpt.1830


In [150]:
pmids = df['PMID'].tolist()

In [151]:
len(pmids)

139

In [153]:
def get_article_data(pmid):
    data = {"PMID": str(pmid)}
    
    try:
        article = fetch.article_by_pmid(pmid)
        
        data.update({
            "Title": article.title,
            "Abstract": article.abstract,
            "Author": article.authors,
            "Year": article.year,
            "Volume": article.volume,
            "Issue": article.issue,
            "Journal": article.journal,
            "Citation": article.citation,
            "Link": f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/",
            "EutilsNCBIError": 0
        })

    except Exception:
        data["AnyError"] = 1

    return data


In [154]:
collected_data = []

for pmid in pmids:
    data = get_article_data(pmid)
    collected_data.append(data)

df = pd.DataFrame(collected_data)

columns_to_fill = ["Title", "Abstract", "Author", "Year", "Volume", "Issue", "Journal", "Citation", "Link"]
df.loc[df["EutilsNCBIError"] == 1, columns_to_fill] = np.nan


In [155]:
df = pd.DataFrame(collected_data)

In [161]:
df.tail(1)

,PMID,Title,Abstract,Author,Year,Volume,Issue,Journal,Citation,Link,EutilsNCBIError
138,29328959,Impact of individual and neighborhood factors ...,BACKGROUND: We addressed the hypothesis that i...,"[DeRouen MC, Schupp CW, Koo J, Yang J, Hertz A...",2018,53,None,Cancer Epidemiol,"DeRouen MC, et al. Impact of individual and ne...",https://pubmed.ncbi.nlm.nih.gov/29328959/,0


In [167]:
pdf_url_or_reason = []
pdf_url_available = []

In [168]:
for pmid in pmids:

  src = FindIt(pmid)

  if src.url is None:
    pdf_url_or_reason.append(src.reason)
    pdf_url_available.append(0)

  else: 
    pdf_url_or_reason.append(src.url)
    pdf_url_available.append(1)
    response = requests.get(src.url, allow_redirects=True)

    with open(os.path.join(save_directory, f"{pmid}.pdf"), 'wb') as f:
      f.write(response.content)

In [169]:
len(pdf_url_available)

139

In [170]:
df["pdf-url"] = pdf_url_or_reason
df["pdf_url_available"] = pdf_url_available

In [171]:
df.to_csv(f'{csv_data}_articledetails.csv')

In [174]:
save_directory

'pdfs_csv-cpic-set'

In [175]:
!ls -l "pdfs_csv-cpic-set" | grep -v '^d' | wc -l

110


other resources for ideas how to extend my code

https://github.com/billgreenwald/Pubmed-Batch-Download

https://github.com/ddomingof/PubMed2PDF